In [119]:
from torchsummary import summary

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision import models
from torchvision import utils

import numpy as np

In [120]:
USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

using device: cuda


In [169]:
NUM_TRAIN = 49000

transform_train = T.Compose([
                      T.Resize(256), T.CenterCrop(224),
                      # T.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),
                      T.ToTensor(), 
                      T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                      ])

transform_test = T.Compose([
                      T.ToTensor(), 
                      T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                      ])

cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform_train)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform_train)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform_test)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [170]:
!pip install torchsummary

In [171]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc

In [172]:
def train(model, optimizer, scheduler, epochs=1):
    model = model.to(device=device)
    for e in range(epochs):
        print(f"--------------------  Epoch: {e+1}  --------------------")
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % print_every == 0:
                print('Epoch %d' % (e+1))
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                val_acc = check_accuracy(loader_val, model)
                print()

        scheduler.step(val_acc)

In [173]:
model = models.resnet101(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 10)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1, bias=False)
model = model.to(device)
print(model.conv1)
summary(model, (3, 32, 32))

Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           1,728
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13      

In [174]:
learning_rate = 1e-3
momentum=0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [175]:
# scheduler = torch.optim.lr_scheduler.LambdaLR(
#     optimizer=optimizer, 
#     lr_lambda=lambda epoch: 0.95**epoch, 
#     last_epoch=-1, 
#     verbose=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, verbose=True, mode='max', patience=5)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


In [ ]:
train(model, optimizer, scheduler, epochs=10)

--------------------  Epoch: 1  --------------------
Epoch 1
Iteration 0, loss = 2.3484
Checking accuracy on validation set
Got 81 / 1000 correct (8.10)

Epoch 1
Iteration 100, loss = 1.2776
Checking accuracy on validation set
Got 566 / 1000 correct (56.60)

Epoch 1
Iteration 200, loss = 0.8850
Checking accuracy on validation set
Got 756 / 1000 correct (75.60)

Epoch 1
Iteration 300, loss = 0.4944
Checking accuracy on validation set
Got 814 / 1000 correct (81.40)

Epoch 1
Iteration 400, loss = 0.4195
Checking accuracy on validation set
Got 843 / 1000 correct (84.30)

Epoch 1
Iteration 500, loss = 0.3590
Checking accuracy on validation set
Got 875 / 1000 correct (87.50)

Epoch 1
Iteration 600, loss = 0.3756
Checking accuracy on validation set
Got 898 / 1000 correct (89.80)

Epoch 1
Iteration 700, loss = 0.3423
Checking accuracy on validation set
Got 915 / 1000 correct (91.50)

--------------------  Epoch: 2  --------------------
Epoch 2
Iteration 0, loss = 0.2445
Checking accuracy on va

In [ ]:
best_model = model
test_acc = check_accuracy(loader_test, best_model)